## Importing all necessary packages

In [26]:
import pandas as pd
import shutil
from os import listdir, mkdir, path, remove
from pathlib import Path
import sys
from email.parser import HeaderParser
import collections
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import time
from collections import Counter
import mailbox

In [27]:
with open('features.txt', 'r') as f:
    columns = [line.strip() for line in f.readlines()]

## Parsing Mbox files

In [28]:
list_of_rows = []

def addEmailsToDict(email_list):
    global list_of_rows 
    # The label for phishing, set to '1'
    label = 1
    
    # Read the full email content
    for email in email_list:
        row_dict = {}
        # Parse recieved field
        received_list = email.get_all('received')
        hops = 0
        if received_list is not None:
            hops = len(received_list)
            col_name_recieved = 'received'

            for inx, received_field in enumerate(received_list):
                col = col_name_recieved + str(inx+1)
                row_dict[col] = received_field


        # Make everything lowercase to avoid issues
        features_lower_case = [x.lower() for x in email.keys()]

        # Parse everything else
        new_row = dict(zip(features_lower_case, email.values()))
        new_row['hops'] = hops
        new_row['body'] = email.get_payload(decode=True)


        for key,value in new_row.items():
            if key in columns:
                row_dict['label'] = label
                row_dict[key] = value
                
        list_of_rows.append(row_dict)
    
def parseMboxFiles(direcrotry, files):
    global list_of_rows
    
    for file in files:
        file_path = direcrotry + file
        print("parsing " + file)
        mboxfile = mailbox.mbox(file_path)
        addEmailsToDict(mboxfile)
        
    df = pd.DataFrame(list_of_rows)
    
    final_columns = []
    for col in columns:
        if col in df.columns:
            final_columns.append(col)

    df = df[final_columns]

    # Output the dataframe to a .csv file
    if path.exists(directory + 'csv/nazario-mbox.csv',):
        remove(directory + 'csv/nazario-mbox.csv')
        df.to_csv(directory + 'csv/nazario-mbox.csv', index=False)
        print('dataset was built at: ' + directory + 'csv/nazario-text.csv' )
        
        
    else:
        df.to_csv(directory + 'csv/nazario-mbox.csv', index=False)
        print('dataset was built at: ' + directory + 'csv/nazario-text.csv' )
        



In [29]:
directory = '../datasets/nazario-phishing-corpus/'
files = [x for x in listdir(directory) if not x.startswith('.') and x.endswith('.mbox') and path.isfile(directory+x) ]
parseMboxFiles(directory, files)

parsing phishing1.mbox
parsing phishing3.mbox
parsing phishing2.mbox
parsing phishing0.mbox
parsing 20051114.mbox
dataset was built at: ../datasets/nazario-phishing-corpus/csv/nazario-text.csv


## Parsing txt files

In [30]:
parser = HeaderParser()
list_of_rows = []

def addEmailsToDict2(email_list):
    global parser, list_of_rows
    
    # The label for phishing, set to '2'
    label = 1
    
    
    # Read the full email content
    for email in email_list:
        row_dict = {}
        
        # Parse the email content
        h = parser.parsestr(email)

        # Parse recieved field
        received_list = h.get_all('received')
        hops = 0
        if received_list is not None:
            hops = len(received_list)
            col_name_recieved = 'received'

            for inx, received_field in enumerate(received_list):
                col = col_name_recieved + str(inx+1)
                row_dict[col] = received_field


        # Make everything lowercase to avoid issues
        features_lower_case = [x.lower() for x in h.keys()]

        # Parse everything else
        new_row = dict(zip(features_lower_case, h.values()))
        new_row['hops'] = hops
        new_row['body'] = h.get_payload(decode=True)


        for key,value in new_row.items():
            if key in columns:
                row_dict['label'] = label
                row_dict[key] = value


        list_of_rows.append(row_dict)

def parseTextFiles(directory, files):
    global list_of_rows
    
    for file in files:

        file_path = directory + file
        print("parsing " + file)

        # Read the full email content
        emailStr = ''
        try:
            with open(file_path, encoding='latin_1') as emailFile:
                for line in emailFile:
                    emailStr += line
        except UnicodeDecodeError:
            print('Unicode Error!')

        email_list = emailStr.split('\nFrom jose@monkey.org')
        #print(email_list[1])

        addEmailsToDict2(email_list)

    # Create the dataframe
    df = pd.DataFrame(list_of_rows)

    final_columns = []
    for col in columns:
        if col in df.columns:
            final_columns.append(col)

    df = df[final_columns]

    # Output the dataframe to a .csv file
    if path.exists(directory + 'csv/nazario-text.csv'):
        remove(directory + 'csv/nazario-text.csv')
        df.to_csv(directory + 'csv/nazario-text.csv', index=False)
        print('dataset was built at: ' + directory + 'csv/nazario-text.csv' )
    else:
        df.to_csv(directory + 'csv/nazario-text.csv', index=False)
        print('dataset was built at: ' + directory + 'csv/nazario-text.csv' )
        




In [31]:
directory = '../datasets/nazario-phishing-corpus/'
files = [x for x in listdir(directory) if not x.endswith('.mbox') and path.isfile(directory+x)]
parseTextFiles(directory, files)

parsing phishing-2018
parsing phishing-2015
parsing phishing-2016
parsing phishing-2017
parsing phishing-2019
parsing phishing-2020
parsing phishing-2021
parsing phishing-2022
dataset was built at: ../datasets/nazario-phishing-corpus/csv/nazario-text.csv
